In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
from model_arch.run_train import *
from utils.step_sample import create_named_schedule_sampler
from model_arch.train import TrainLoop
from utils.data import load_data_text
from model_arch.tokenizer import load_tokenizer, load_model_emb
from model_arch.sampling import sampling

from transformers import AutoTokenizer, PreTrainedTokenizerFast, BertTokenizerFast, set_seed
import json, torch
from utils import dist_util
from functools import partial
import pickle
import random
from datetime import datetime

2024-03-07 16:32:59.076686: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-07 16:32:59.076753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-07 16:32:59.078465: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-07 16:32:59.089465: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 16:33:00.639125: W tensorflow/compiler/tf2

In [3]:
dist_util.clear_cache()

In [4]:
lr=0.0001
batch_size=30
val_batch_size=30
microbatch=10
epochs=30_000
eval_interval=10
ema_rate='0.9999' 
schedule_sampler='uniform'
diffusion_steps=1000
noise_schedule='sqrt'
vocab='custom'
use_plm_init='no' # embedding in transformer
vocab_size=0
config_name='bert-base-uncased'
seq_len=128
hidden_t_dim=128
hidden_dim=128
dropout=0.1
seed=10275679
weight_decay=0.0
predict_xstart=True
rescale_timesteps=True
emb_scale_factor=1.0

In [5]:
regular_data_dir='data'
data_player_dir='data/with_player'

# set the data directory
data_dir=regular_data_dir

In [6]:
set_seed(seed)

In [7]:
tokenizer = load_tokenizer('shakespeare', config_name)

shakespeare


In [8]:
model_weight, tokenizer = load_model_emb(hidden_dim, tokenizer)

In [9]:
model_weight

Embedding(30268, 128)

In [10]:
## very very important to set this!!!!!
vocab_size = tokenizer.vocab_size
vocab_size

30268

In [11]:
data = load_data_text(
        batch_size=batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        model_emb=model_weight # use model's weights as init
    )

val = load_data_text(
        batch_size=val_batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        split='valid',
        model_emb=model_weight, # use model's weights as init
    )

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading form the TRAIN set...
### Data samples...
 ['this is the very false gallop of verses why do you infect yourself with them?', 'and fair men call for grace. aaron will have his soul black like his face.'] ['peace, you dull fool! i found them on a tree.', 'o, here i lift this one hand up to heaven, and bow this feeble ruin to the earth if any power pities wretched tears, to that i call! what, wilt thou kneel with me? do, then, dear heart,']
RAM used: 718.34 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 32531
})
RAM used: 737.28 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/32531 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 32531
})
### tokenized_datasets...example [2, 138, 121, 78, 476, 880, 9871, 94, 5833, 329, 144, 89, 2960, 955, 131, 266, 22, 3]
RAM used: 776.12 MB


merge and mask:   0%|          | 0/32531 [00:00<?, ? examples/s]

RAM used: 811.85 MB


...

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 8123
})
### tokenized_datasets...example [2, 714, 37, 9, 749, 243, 23, 550, 85, 317, 31, 241, 125, 100, 208, 10, 6206, 10, 136, 469, 23187, 180, 646, 37, 9, 579, 3437, 10, 85, 1044, 549, 31, 9, 78, 5789, 237, 571, 295, 37, 9, 78, 14784, 12, 3]
RAM used: 846.08 MB


merge and mask:   0%|          | 0/8123 [00:00<?, ? examples/s]

RAM used: 859.91 MB


padding:   0%|          | 0/8123 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 8123
}) padded dataset
RAM used: 874.72 MB
RAM used: 874.72 MB


In [12]:
model, diffusion = create_model_and_diffusion(
                        hidden_t_dim,
                        hidden_dim,
                        vocab_size,
                        config_name,
                        use_plm_init,
                        dropout,
                        diffusion_steps,
                        noise_schedule,
                        predict_xstart,
                        rescale_timesteps,
                    )

In [13]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

91192508

In [14]:
# from torch import nn
# from transformers import BertConfig

# config = BertConfig.from_pretrained("bert-base-uncased")

# for layer in model.input_transformers.layer[-1:]:
#     for module in layer.modules():
#         if isinstance(module, nn.Linear):
#             module.weight.data.normal_(mean=0.0, std=config.initializer_range)
#             if module.bias is not None:
#                 module.bias.data.zero_()
#         elif isinstance(module, nn.Embedding):
#             module.weight.data.normal_(mean=0.0, std=config.initializer_range)
#             if module.padding_idx is not None:
#                 module.weight.data[module.padding_idx].zero_()
#         elif isinstance(module, nn.LayerNorm):
#             module.bias.data.zero_()
#             module.weight.data.fill_(1.0)

In [15]:
schedule_sampler = create_named_schedule_sampler('uniform', diffusion)

model.to(dist_util.dev())

train_loss, val_loss = TrainLoop(
                            model=model,
                            diffusion=diffusion,
                            data=data,
                            batch_size=batch_size,
                            microbatch=microbatch,
                            lr=lr,
                            ema_rate=ema_rate,
                            schedule_sampler=schedule_sampler,
                            weight_decay=weight_decay,
                            epochs=epochs,
                            eval_data=val,
                            eval_interval=eval_interval,
                            warm_up_steps=500,
                            use_llrd=True,
                            llrd_rate=0.9
                        ).run_loop()

dt = datetime.now().strftime("%m%d")
pickle.dump(model, open(f"models/{dt}/final_model_df{diffusion_steps}.pkl", 'wb'))



======== Using Layer-wise Learning Rate Decay with AdamW ========


name: word_embedding.weight, lr: 0.0001
name: lm_head.bias, lr: 0.0001
name: time_embed.0.weight, lr: 0.0001
name: time_embed.0.bias, lr: 0.0001
name: time_embed.2.weight, lr: 0.0001
name: time_embed.2.bias, lr: 0.0001
name: input_up_proj.0.weight, lr: 0.0001
name: input_up_proj.0.bias, lr: 0.0001
name: input_up_proj.2.weight, lr: 0.0001
name: input_up_proj.2.bias, lr: 0.0001
name: input_transformers.layer.0.attention.self.query.weight, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.query.bias, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.key.weight, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.key.bias, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.value.weight, lr: 0.00011111111111111112
name: input_transformers.layer.0.attention.self.value.bias, lr: 0.00011111111111111112
name: input_transformers.la

KeyboardInterrupt: 

In [ ]:
# plot(train_loss, val_loss)

In [ ]:
# param_names = []
# for i, (name, param) in enumerate(model.named_parameters()):
#     param_names.append(name)
#     print(f'{i}: {name} {param.requires_grad}')

In [35]:
# best_model_fp = f'models/results/model_df2000_best_epoch_26150_min_val_loss_0.030700000002980232.pkl'
# with open(best_model_fp, 'rb') as handle:
#     best_model = pickle.load(handle)

# Generating sequences

In [41]:
diffusion_2000 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 2000),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

In [42]:
word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_2000, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=128, 
                                                           show_intermediate_results=False)

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ["My lord, 'tis the pondering of life's meaning that doth occupy my thoughts most gravely.", 'call her forth to me.'] ['', '']
RAM used: 3236.95 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 5
})
RAM used: 3236.95 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/5 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 5
})
### tokenized_datasets...example [2, 105, 203, 10, 9, 334, 78, 14472, 102, 94, 451, 9, 41, 2679, 110, 457, 19200, 105, 1079, 367, 23844, 12, 3]
RAM used: 3238.46 MB


merge and mask:   0%|          | 0/5 [00:00<?, ? examples/s]

RAM used: 3238.47 MB


padding:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 5
}) padded dataset
RAM used: 3238.51 MB
RAM used: 3238.51 MB
### End of reading iteration...


  0%|          | 0/2000 [00:00<?, ?it/s]

Generating 20 sentences takes 5 minutes

In [43]:
word_lst_source

["[CLS] my lord,'tis the pondering of life's meaning that doth occupy my thoughts most gravely. [SEP] [SEP]",
 '[CLS] call her forth to me. [SEP] [SEP]',
 '[CLS] what is your will? [SEP] [SEP]',
 '[CLS] hark, fair sarah! [SEP] [SEP]',
 "[CLS] what dost thou have planned for the morrow's reprieve? [SEP] [SEP]"]

In [44]:
word_lst_recover

['[CLS] my lord. [SEP]',
 '[CLS] she so from a reason now, to wife my wife the wife man no from a most him come the wife [SEP]',
 '[CLS] come to his thing your wife me. [SEP]',
 '[CLS] o, the of pretty, in touch,. [SEP]',
 '[CLS] who lord, she them that. [SEP]']